In [2]:
import torch
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from PIL import Image
import numpy as np
import os

# Set device to cuda
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Download pre-trained model
model = models.vgg16_bn(pretrained=True)

In [3]:
# Create a list of paths to the image with appropriate labels.
def get_img_paths_with_labels(data_dir, label):
    image_paths = []
    labels = []
    for file_name in os.listdir(data_dir):
        file_path = os.path.join(data_dir, file_name)
        if os.path.isfile(file_path):
            image_paths.append(file_path)
            labels.append(label)
    return image_paths, labels

In [4]:
class CustomImageDataset(Dataset):
    """
    Custom dataset for loading images and their corresponding labels.

    Args:
        image_paths (list of str): List of file paths to the images.
        labels (list of int): List of labels corresponding to the images.
        transform (callable, optional): Optional transform to be applied to each image.
    """
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB") # Open image as RGB
        label = self.labels[idx]

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
# Paths to the folders with data
data_dir = {
    "train_1": os.path.join(os.getenv("CAR_OR_NOT"), "Car_photos_train"),
    "train_0": os.path.join(os.getenv("CAR_OR_NOT"), "Not_car_photos_train"),
    "test_1":  os.path.join(os.getenv("CAR_OR_NOT"), "Car_photos_test"),
    "test_0":  os.path.join(os.getenv("CAR_OR_NOT"), "Not_car_photos_test"),
}

# Load images from directories
train_car_paths, train_car_labels = get_img_paths_with_labels(
    data_dir["train_1"], 1
)
train_not_car_paths, train_not_car_labels = get_img_paths_with_labels(
    data_dir["train_0"], 0
)

test_car_paths, test_car_labels = get_img_paths_with_labels(
    data_dir["test_1"], 1
)
test_not_car_paths, test_not_car_labels = get_img_paths_with_labels(
    data_dir["test_0"], 0
)

# Combine the paths and labels
train_image_paths = train_car_paths + train_not_car_paths
train_labels = train_car_labels + train_not_car_labels

test_image_paths = test_car_paths + test_not_car_paths
test_labels = test_car_labels + test_not_car_labels

In [58]:
def save_transformed_image(tensor_image, output_path):
    """
    Save a tensor image as a .png file after transforming it to a PIL image.

    Args:
        tensor_image (Tensor): The image tensor to save.
        output_path (str): The path where the image will be saved.
    """
    # Convert the tensor image to a numpy array in the format (H, W, C)
    np_image = tensor_image.permute(1, 2, 0).cpu().numpy()
    # Rescale the image to the range [0, 255] and convert to uint8
    np_image = (np_image * 255).astype(np.uint8)
    # Convert to PIL image and save
    pil_image = Image.fromarray(np_image)
    pil_image.save(output_path)

# Define transformation to convert images to tensors
preprocess_transform = transforms.Compose([
    transforms.ToTensor()
])

# Create training dataset with preprocessing transformations
train_dataset_preprocess = CustomImageDataset(
    train_image_paths, train_labels, transform=preprocess_transform
)

# Initialize DataLoader for the training dataset
loader = DataLoader(
    train_dataset_preprocess, batch_size=64, shuffle=False, num_workers=0
)

# Calculate mean and standard deviation for normalization
mean = 0.0
std = 0.0
total_images = 0

# Iterate over the data to compute mean and std for each channel
for images, _ in loader:
    images = images.to(device)
    batch_samples = images.size(0)  # Number of images in the batch
    images = images.view(batch_samples, images.size(1), -1)  # Flatten the images
    mean += images.mean(dim=2).sum(dim=0)  # Sum over batch
    std += images.std(dim=2).sum(dim=0)  # Sum over batch
    total_images += batch_samples

# Compute the final mean and std
mean /= total_images
std /= total_images

print(f"Mean: {mean}, Std: {std}")

# Define transformations for training and testing datasets
train_transformations = transforms.Compose([
    transforms.Resize(size=(224, 224)),  # Resize images to 224x224
    transforms.RandomRotation(degrees=45),  # Random rotation
    transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip
    transforms.RandomVerticalFlip(p=0.05),  # Random vertical flip with low probability
    transforms.RandomGrayscale(p=0.33),  # Random grayscale with a 33% chance
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=mean, std=std),  # Normalize using computed mean and std
])

test_transformations = transforms.Compose([
    transforms.Resize(size=(224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=mean, std=std),  # Normalize using computed mean and std
])

# Create the training and testing datasets with transformations
train_dataset = CustomImageDataset(
    train_image_paths, train_labels, transform=train_transformations
)
test_dataset = CustomImageDataset(
    test_image_paths, test_labels, transform=test_transformations
)

# Initialize DataLoader for training and testing datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

Mean: tensor([0.4297, 0.4375, 0.4382], device='cuda:0'), Std: tensor([0.2463, 0.2426, 0.2417], device='cuda:0')


In [59]:
for param in model.features.parameters():
    param.requires_grad = False  # Freeze feature extraction layers

# Add an output layer to have only to classes
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)

# Move model to the GPU
model = model.to(device)

# Add optimizer and loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters())


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    """
    Train the model on the training dataset and validate it on the validation dataset.

    Args:
        model (nn.Module): The model to train.
        train_loader (DataLoader): DataLoader for the training dataset.
        val_loader (DataLoader): DataLoader for the validation dataset.
        criterion (nn.Module): The loss function.
        optimizer (torch.optim.Optimizer): The optimizer.
        num_epochs (int): Number of epochs to train the model.
    """
    for epoch in range(num_epochs):
        model.train() # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        # Training loop with progress bar (tqdm)
        with tqdm(
            total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"
        ) as pbar:
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()  # Zero the gradients before backpropagation

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward pass and optimization
                loss.backward()
                optimizer.step()

                # Calculate the loss and accuracy for the current batch
                running_loss += loss.item() * images.size(0)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

                # Update the progress bar with current batch results
                pbar.update(1)
                pbar.set_postfix(
                    {"Loss": loss.item(), "Accuracy": 100.0 * correct / total}
                )

        # Calculate and print training loss and accuracy after each epoch
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = 100.0 * correct / total
        print(
            f"\nEpoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%"
        )

        # Validate the model after each epoch
        validate_model(model, val_loader, criterion)


def validate_model(model, val_loader, criterion):
    """
    Validate the model on the validation dataset.

    Args:
        model (nn.Module): The model to validate.
        val_loader (DataLoader): DataLoader for the validation dataset.
        criterion (nn.Module): The loss function.
    """
    model.eval() # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    # Validation loop with progress bar (tqdm)
    with tqdm(total=len(val_loader), desc="Validation", unit="batch") as pbar:
        with torch.no_grad(): # No gradient computation during validation
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

                # Update the progress bar with current batch results
                pbar.update(1)
                pbar.set_postfix(
                    {"Val Loss": loss.item(), "Val Accuracy": 100.0 * correct / total}
                )

    # Calculate and print validation loss and accuracy after each epoch
    val_loss = val_loss / len(val_loader.dataset)
    val_acc = 100.0 * correct / total
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

In [60]:
train_model(model, train_loader, test_loader, criterion, optimizer)

Epoch 1/10: 100%|██████████| 109/109 [00:28<00:00,  3.81batch/s, Loss=0.0242, Accuracy=90.5] 



Epoch [1/10], Loss: 0.3594, Accuracy: 90.53%


Validation: 100%|██████████| 32/32 [00:20<00:00,  1.57batch/s, Val Loss=5.81e-7, Val Accuracy=98]   


Validation Loss: 0.0817, Validation Accuracy: 98.00%


Epoch 2/10: 100%|██████████| 109/109 [00:35<00:00,  3.09batch/s, Loss=0.00376, Accuracy=96]  


...

Validation Loss: 0.0676, Validation Accuracy: 98.30%


Epoch 10/10: 100%|██████████| 109/109 [01:09<00:00,  1.58batch/s, Loss=0.203, Accuracy=97.2]  



Epoch [10/10], Loss: 0.1044, Accuracy: 97.23%


Validation: 100%|██████████| 32/32 [00:17<00:00,  1.86batch/s, Val Loss=0, Val Accuracy=98.1]       

Validation Loss: 0.0855, Validation Accuracy: 98.10%


In [ ]:
torch.save(
    model.state_dict(),
    "C:\\Users\\jakub\\PycharmProjects\\Car_Finder\\Model\\Weights\\car_or_not_model.pth",
)
print(f"Model saved")

Model saved
